In [8]:
import pandas as pd
from collections import namedtuple
from pprint import pprint

pd.set_option('display.max_columns', 200)

In [9]:
CellInfo = namedtuple('CellInfo', 'PCI, ARFCN, RSRP, RSRQ')

def set_dtypes(df):
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=8)
    df['Type_ID'] = df['Type_ID'].astype('category')
    df['Serv_Cell_Index'] = df['Serv_Cell_Index'].astype('category')
    for tag in df.columns:
        if df[tag].dtype == 'float64':
            df[tag] = df[tag].astype('float32')
        elif df[tag].dtype == 'int64':
            df[tag] = df[tag].astype('uint16')
    return df

def concat_cell_info(df_lte, df_nr):
    df_lte = df_lte.rename(columns={
        'type_id': 'Type_ID',
        'RSRP(dBm)': 'RSRP',
        'RSRQ(dB)': 'RSRQ',
        'Serving Cell Index': 'Serv_Cell_Index',
        'Number of Neighbor Cells': 'Num_Neigh_Cells',
        'Number of Detected Cells': 'Num_Cells',
    })
    df_lte = df_lte.reindex(['Timestamp', 'Type_ID', 'PCI', 'RSRP', 'RSRQ', 'Serv_Cell_Index', 'EARFCN', 'Raster_ARFCN',
                    'Num_Cells', 'Num_Neigh_Cells', 'Serv_Cell_Pos', 'Serv_Cell_PCI',
                    'PCI0', 'RSRP0', 'RSRQ0', *df_lte.columns.to_list()[df_lte.columns.get_loc('PCI1'):]], axis=1)
    df_nr = df_nr.rename(columns={
        'type_id': 'Type_ID',
        'Raster ARFCN': 'Raster_ARFCN',
        'Serving Cell Index': 'Serv_Cell_Pos',
        'Serving Cell PCI': 'Serv_Cell_PCI',
        'Num Cells': 'Num_Cells',
    })
    df_nr = df_nr.reindex(['Timestamp', 'Type_ID', 'PCI', 'RSRP', 'RSRQ', 'Serv_Cell_Index', 'EARFCN', 'Raster_ARFCN',
                    'Num_Cells', 'Num_Neigh_Cells', 'Serv_Cell_Pos', 'Serv_Cell_PCI',
                    'PCI0', 'RSRP0', 'RSRQ0', *df_nr.columns.to_list()[df_nr.columns.get_loc('PCI1'):]], axis=1)
    df = pd.concat([df_lte, df_nr])
    # df = set_dtypes(df)
    # df['Timestamp'] = df['Timestamp'] + pd.Timedelta(hours=8)
    df = df.sort_values(by=['Timestamp'])
    df = df.reset_index(drop=True)
    return df

def get_info(row):
    row_dict = {
        'Timestamp': None,
        'RAT': None,
        'Serv_Cell_Index': None,
        'Serv_Cell': None,
        'Num_Neigh_Cells': None,
        'Neigh_Cells': []
    }
    if row.Type_ID == 'LTE_PHY_Connected_Mode_Intra_Freq_Meas':
        row_dict['Timestamp'] = row.Timestamp
        row_dict['RAT'] = 'LTE'
        row_dict['Serv_Cell_Index'] = row.Serv_Cell_Index
        row_dict['Serv_Cell'] = CellInfo(int(row.PCI), int(row.EARFCN), row.RSRP, row.RSRQ)
        row_dict['Num_Neigh_Cells'] = int(row.Num_Neigh_Cells)
        for i in range(1, int(row.Num_Neigh_Cells)+1):
            row_dict['Neigh_Cells'].append(CellInfo(int(row[f'PCI{i}']), int(row.EARFCN), row[f'RSRP{i}'], row[f'RSRQ{i}']))
    elif row.Type_ID == '5G_NR_ML1_Searcher_Measurement_Database_Update_Ext':
        row_dict['Timestamp'] = row.Timestamp
        row_dict['RAT'] = 'NR'
        row_dict['Num_Neigh_Cells'] = int(row.Num_Cells)
        pos = int(row.Serv_Cell_Pos)
        if pos != 255:
            row_dict['Num_Neigh_Cells'] -= 1
            row_dict['Serv_Cell'] = CellInfo(int(row[f'PCI{pos}']), int(row.Raster_ARFCN), row[f'RSRP{pos}'], row[f'RSRQ{pos}'])
            row_dict['Serv_Cell_Index'] = 'PSCell'
        for i in range(int(row.Num_Cells)):
            if i == pos:
                continue
            row_dict['Neigh_Cells'].append(CellInfo(int(row[f'PCI{i}']), int(row.Raster_ARFCN), row[f'RSRP{i}'], row[f'RSRQ{i}']))
    return row_dict

In [3]:
%%timeit
df_lte = pd.read_csv("/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/diag_log_qc01_2023-02-04_16-13-28_ml1.csv")
df_nr = pd.read_csv("/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/diag_log_qc01_2023-02-04_16-13-28_nr_ml1.csv")
df = concat_cell_info(df_lte, df_nr)

26.8 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
df_lte = pd.read_csv("/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/diag_log_qc01_2023-02-04_16-13-28_ml1.csv")
df_nr = pd.read_csv("/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/diag_log_qc01_2023-02-04_16-13-28_nr_ml1.csv")
df = concat_cell_info(df_lte, df_nr)

In [5]:
%%timeit
row = df.iloc[1]
get_info(row)
row = df.iloc[2]
get_info(row)
row = df.iloc[16]
get_info(row)

567 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
row = df.iloc[1]
pprint(get_info(row), sort_dicts=False)
row = df.iloc[2]
pprint(get_info(row), sort_dicts=False)
row = df.iloc[16]
pprint(get_info(row), sort_dicts=False)

{'Timestamp': Timestamp('2023-02-04 16:13:49.977583'),
 'RAT': 'NR',
 'Serv_Cell_Index': None,
 'Serv_Cell': None,
 'Num_Neigh_Cells': 3,
 'Neigh_Cells': [CellInfo(PCI=35, ARFCN=631000, RSRP=-78.508, RSRQ=-11.5),
                 CellInfo(PCI=160, ARFCN=631000, RSRP=-86.008, RSRQ=-17.5),
                 CellInfo(PCI=152, ARFCN=631000, RSRP=-90.508, RSRQ=-19.5)]}
{'Timestamp': Timestamp('2023-02-04 16:13:50.051895'),
 'RAT': 'LTE',
 'Serv_Cell_Index': 'PCell',
 'Serv_Cell': CellInfo(PCI=152, ARFCN=1750, RSRP=-77.8125, RSRQ=-10.25),
 'Num_Neigh_Cells': 2,
 'Neigh_Cells': [CellInfo(PCI=400, ARFCN=1750, RSRP=-90.875, RSRQ=-18.9375),
                 CellInfo(PCI=27, ARFCN=1750, RSRP=-85.0625, RSRQ=-16.5625)]}
{'Timestamp': Timestamp('2023-02-04 16:13:52.519282'),
 'RAT': 'NR',
 'Serv_Cell_Index': 'PSCell',
 'Serv_Cell': CellInfo(PCI=35, ARFCN=631000, RSRP=-75.273, RSRQ=-11.039),
 'Num_Neigh_Cells': 2,
 'Neigh_Cells': [CellInfo(PCI=160, ARFCN=631000, RSRP=-80.633, RSRQ=-16.328),
          

In [11]:
df

,Timestamp,Type_ID,PCI,RSRP,RSRQ,Serv_Cell_Index,EARFCN,Raster_ARFCN,Num_Cells,Num_Neigh_Cells,Serv_Cell_Pos,Serv_Cell_PCI,PCI0,RSRP0,RSRQ0,PCI1,RSRP1,RSRQ1,PCI2,RSRP2,RSRQ2,PCI3,RSRP3,RSRQ3,PCI4,RSRP4,RSRQ4,PCI5,RSRP5,RSRQ5,PCI6,RSRP6,RSRQ6,PCI7,RSRP7,RSRQ7,PCI8,RSRP8,RSRQ8
0,2023-02-04 08:13:49.872964,LTE_PHY_Connected_Mode_Intra_Freq_Meas,152.0,-77.8125,-10.2500,PCell,1750.0,NaN,0,1.0,NaN,NaN,NaN,NaN,NaN,27.0,-87.5000,-17.4375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-04 08:13:49.977583,5G_NR_ML1_Searcher_Measurement_Database_Update...,NaN,NaN,NaN,NaN,NaN,631000.0,3,NaN,255.0,65535.0,35.0,-78.508,-11.500,160.0,-86.0080,-17.5000,152.0,-90.5080,-19.5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-04 08:13:50.051895,LTE_PHY_Connected_Mode_Intra_Freq_Meas,152.0,-77.8125,-10.2500,PCell,1750.0,NaN,0,2.0,NaN,NaN,NaN,NaN,NaN,400.0,-90.8750,-18.9375,27.0,-85.0625,-16.5625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-04 08:13:50.088433,5G_NR_ML1_Searcher_Measurement_Database_Update...,NaN,NaN,NaN,NaN,NaN,631000.0,3,NaN,0.0,35.0,35.0,-78.508,-11.500,160.0,-86.0080,-17.5000,152.0,-90.5080,-19.5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-04 08:13:50.256852,LTE_PHY_Connected_Mode_Intra_Freq_Meas,152.0,-77.7500,-9.8125,PCell,1750.0,NaN,0,2.0,NaN,NaN,NaN,NaN,NaN,27.0,-84.6250,-14.7500,400.0,-92.3750,-18.9375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851,2023-02-04 08:25:37.846313,LTE_PHY_Connected_Mode_Intra_Freq_Meas,362.0,-86.3750,-9.7500,PCell,1750.0,NaN,0,5.0,NaN,NaN,NaN,NaN,NaN,350.0,-95.8125,-14.6875,204.0,-90.1250,-13.5625,422.0,-96.750,-18.0625,370.0,-95.9375,-18.3125,174.0,-100.3125,-19.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4852,2023-02-04 08:25:38.036594,5G_NR_ML1_Searcher_Measurement_Database_Update...,NaN,NaN,NaN,NaN,NaN,631000.0,3,NaN,0.0,350.0,350.0,-85.062,-13.703,174.0,-87.9450,-16.1880,362.0,-91.6170,-18.8590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4853,2023-02-04 08:25:38.356597,5G_NR_ML1_Searcher_Measurement_Database_Update...,NaN,NaN,NaN,NaN,NaN,631000.0,4,NaN,0.0,350.0,350.0,-84.914,-13.055,174.0,-89.7110,-17.4690,362.0,-90.6640,-18.9380,422.0,-92.500,-21.0080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4854,2023-02-04 08:25:38.676595,5G_NR_ML1_Searcher_Measurement_Database_Update...,NaN,NaN,NaN,NaN,NaN,631000.0,4,NaN,0.0,350.0,350.0,-84.648,-13.250,174.0,-90.8280,-18.0470,362.0,-90.9610,-18.1410,422.0,-89.492,-17.9610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.dtypes

Timestamp           object
Type_ID             object
PCI                float64
RSRP               float64
RSRQ               float64
Serv_Cell_Index     object
EARFCN             float64
Raster_ARFCN       float64
Num_Cells            int64
Num_Neigh_Cells    float64
Serv_Cell_Pos      float64
Serv_Cell_PCI      float64
PCI0               float64
RSRP0              float64
RSRQ0              float64
PCI1               float64
RSRP1              float64
RSRQ1              float64
PCI2               float64
RSRP2              float64
RSRQ2              float64
PCI3               float64
RSRP3              float64
RSRQ3              float64
PCI4               float64
RSRP4              float64
RSRQ4              float64
PCI5               float64
RSRP5              float64
RSRQ5              float64
PCI6               float64
RSRP6              float64
RSRQ6              float64
PCI7               float64
RSRP7              float64
RSRQ7              float64
PCI8               float64
R

In [13]:
df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=8)
df['Type_ID'] = df['Type_ID'].astype('category')
df['Serv_Cell_Index'] = df['Serv_Cell_Index'].astype('category')
df['EARFCN'] = df['EARFCN'].astype('Int32')
df['Raster_ARFCN'] = df['Raster_ARFCN'].astype('Int32')
df['Num_Cells'] = df['Num_Cells'].astype('UInt8')
df['Num_Neigh_Cells'] = df['Num_Neigh_Cells'].astype('UInt8')
df['Serv_Cell_Pos'] = df['Serv_Cell_Pos'].astype('UInt8')
df['Serv_Cell_PCI'] = df['Serv_Cell_PCI'].astype('UInt16')
for tag in df.columns:
    if tag.startswith('PCI'):
        df[tag] = df[tag].astype('Int32')
    if tag.startswith(('RSRP','RSRQ')):
        df[tag] = df[tag].astype('float32')

In [14]:
df.dtypes

Timestamp          datetime64[ns]
Type_ID                  category
PCI                         Int32
RSRP                      float32
RSRQ                      float32
Serv_Cell_Index          category
EARFCN                      Int32
Raster_ARFCN                Int32
Num_Cells                   UInt8
Num_Neigh_Cells             UInt8
Serv_Cell_Pos               UInt8
Serv_Cell_PCI              UInt16
PCI0                        Int32
RSRP0                     float32
RSRQ0                     float32
PCI1                        Int32
RSRP1                     float32
RSRQ1                     float32
PCI2                        Int32
RSRP2                     float32
RSRQ2                     float32
PCI3                        Int32
RSRP3                     float32
RSRQ3                     float32
PCI4                        Int32
RSRP4                     float32
RSRQ4                     float32
PCI5                        Int32
RSRP5                     float32
RSRQ5         